In [315]:
import requests
import json
import pandas as pd
import numpy as np
from collections import Counter

#### Create a playlist

In [316]:
# create a new playlist to store those recommendations
def create_plst(user_id, token, uris, name, description, public = False):
    endpoint_url = f"https://api.spotify.com/v1/users/{user_id}/playlists"
    request_body = json.dumps({
        "name": name,
        "description": description,
        "public": public })
    # create an empty new playlist
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    url = response.json()['external_urls']['spotify']
    if response.status_code == 201:
        print('Playlist {} is successfully created!'.format(name))
        
    # fill the new playlist with the recommendations
    playlist_id = response.json()['id']
    endpoint_url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    request_body = json.dumps({ "uris" : uris })
    response = requests.post(url = endpoint_url, data = request_body, headers={"Content-Type":"application/json", 
                        "Authorization":f"Bearer {token}"})
    if response.status_code == 201:
        print('Playlist {} is successfully filled with recommendations!'.format(name))
        print(f'Your playlist is ready at {url}')

### Get a user's recently played songs
- `limit`: The maximum number of items to return. Default: `20`. Minimum: `1`. Maximum: `50`.
- `after`: A Unix timestamp in milliseconds. Returns all items after (but not including) this cursor position. If `after` is specified, `before` must not be specified.
- `before`: A Unix timestamp in milliseconds. Returns all items before (but not including) this cursor position. If `before` is specified, `after` must not be specified.

In [317]:
def get_recent_songs(user_id, token, **kwargs):
    query = "https://api.spotify.com/v1/me/player/recently-played"
    
    if len(kwargs.keys()) > 0:
        query += '?'
        lst = [str(x[0]) + '=' + str(x[1]) for x in zip(kwargs.keys(), kwargs.values())]
        query += '&'.join(lst) ## add all parameters from kwargs 
    #print(query)

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    print("status:", response.status_code)
    song_ids = [json_response['items'][i]['track']['id'] for i in range(len(json_response['items']))]
    return song_ids

### Get a User's Top Artists and Tracks
- `time_range`: Over what time frame the affinities are computed. Valid values: `long_term` (calculated from several years of data and including all new data as it becomes available), `medium_term` (approximately last 6 months), `short_term` (approximately last 4 weeks). Default: `medium_term`
- `limit`: The maximum number of items to return. Default: `20`. Minimum: `1`. Maximum: `50`.
- `offset`: The index of the first entity to return. Default: `0` (i.e., the first track). Use with limit to get the next set of entities.

**Note: songs that have been played for once will not be considered as a top song.**

In [318]:
def get_top_songs(user_id, token, **kwargs):
    query = "https://api.spotify.com/v1/me/top/tracks?"
    
    if len(kwargs.keys()) > 0:
        lst = [str(x[0]) + '=' + str(x[1]) for x in zip(kwargs.keys(), kwargs.values())]
        query += '&'.join(lst) ## add all parameters from kwargs 
    #print(query)

    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    print("status:", response.status_code)
    song_ids = [json_response['items'][i]['id'] for i in range(len(json_response['items']))]
    return song_ids

### Get information about a song

- `market`: The market you’d like to request.

In [319]:
def get_song(user_id, token, song_id, market):
    
    ## get audio features of the song/track
    song_url = "https://api.spotify.com/v1/audio-features/"
    query = f'{song_url}{song_id}'
    success = False
    while not success:
        try:
            response = requests.get(query, 
                           headers={"Content-Type":"application/json", 
                                    "Authorization":f"Bearer {token}"})
            json_response = response.json()
            df_response = pd.json_normalize(json_response)
            success = True
        except Exception as e:
            pass
            
    ## get album information and the first artist
    base_url = "https://api.spotify.com/v1/tracks/"
    query = f'{base_url}{song_id}?market={market}'
    success = False
    while not success:
        try:
            response = requests.get(query, 
                           headers={"Accept": "application/json",
                                    "Content-Type":"application/json", 
                                    "Authorization":f"Bearer {token}"})
            json_response = response.json()
            album_id = json_response['album']['id'] if 'album' in json_response.keys() else None
            artist_id = json_response['artists'][0]['id'] if 'artists' in json_response.keys() else None
            success = True
        except Exception as e:
            pass
            
    if 'genre' in df_response.columns and df_response['genre'] is not None and len(df_response['genre']) > 0:
        return df_response.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1).set_index('id')
    else:            
        ## get genre of the album
        base_url = "https://api.spotify.com/v1/albums/"
        query = f'{base_url}{album_id}?market={market}'
        success = False
        while not success:
            try:
                response = requests.get(query, 
                               headers={"Accept": "application/json",
                                        "Content-Type":"application/json", 
                                        "Authorization":f"Bearer {token}"})
                json_response = response.json()
                genre = json_response['genres'] if 'genres' in json_response.keys() else None
                success = True
            except Exception as e:
                pass

        if genre is None or len(genre) < 1:
            ## get genre of the artist
            base_url = "https://api.spotify.com/v1/artists/"
            query = f'{base_url}{artist_id}'
            success = False
            while not success:
                try:
                    response = requests.get(query, 
                                   headers={"Accept": "application/json",
                                            "Content-Type":"application/json", 
                                            "Authorization":f"Bearer {token}"})
                    json_response = response.json()
                    genre = json_response['genres'] if 'genres' in json_response.keys() else None
                    success = True
                except Exception as e:
                    pass

    df_response['artist_id'] = artist_id if artist_id is not None and len(artist_id) >0 else None
    df_response['albuma_id'] = album_id if album_id is not None and len(album_id) >0 else None
    df_response['genre'] = [genre] if genre is not None and len(genre) > 0 else None
    try:
        return df_response.drop(['type', 'uri', 'track_href', 'analysis_url'], axis=1).set_index('id')
    except Exception:
        print("Cannot find features about this song: ", song_id)

## Generate a target playlist

- `workout_plan`: Workout plan. For example, `workout_plan=[(0, 0.2), (8, 0.9), (20, 0.4)]` means that in the first 8 minutes, to increase energy from 0.2 to 0.9 and then decrease energy to 0.4 in the next 12 minutes.
- `history_args`: Parameters used to extract recently played songs. For example, `history_args = {'limit': 15, 'after': 28885117}`.
- `start_song`: The first song in the target playlist. You can input the id of a song.
- `end_song`: The last song in the target playlist. You can input the id of a song.
- `pars_weight`: When calculating the difference between songs, we use a weighted average of the difference. 
- `genre`: If it is not None, we will filter songs based on genre.
- `recently`: If it is True, we will use recently played songs; otherwise, we will use top songs.

**Note 1. if `recently=True`, `history_args` can accept "limit", "after" (or "before"); otherwise, `history_args` can accept "limit", "time_range", "offset".** 

**Note 2. The required scopes of token is different for "top" and "recently played" songs.**

In [516]:
class Error(Exception):
    pass

class SongError(Error):

    def __init__(self, message):
        self.message = message
        
def flatten(x):
    return sum([flatten(xx) if isinstance(xx, list) else [xx] for xx in x], [])


def generate_target(user_id, token, market, workout_plan, history_args = {}, start_song = None, end_song = None,
                    pars_weight = {'energy': 0.6, 'duration_ms': 0.4},
                    genre = None, recently = True, MAX_songs = 100, time_interval = 15):
    
    convert = lambda x: 60 * 1000 * x ## convert to ms
    reconvert = lambda x: x / 60 / 1000 ## convert to minute
    intersection = lambda lst1, lst2: [value for value in lst1 if value in lst2]
    def get_songs(my_offset):
        ## Note: for recently played songs, it cannot iterate to get more songs right now, it can be improved later.
        if recently:
            song_ids = get_recent_songs(user_id, token, **history_args)
        else:
            offset = max(my_offset, history_args.get('offset', 0))
            history_args['offset'] = offset
            song_ids = get_top_songs(user_id, token, **history_args)
        if len(song_ids) > 0:
            song_ids = set(song_ids) ## remove duplicate songs
            song_df = pd.concat([get_song(user_id, token, id, market) for id in song_ids])
            return song_df
        else:
            return None
    
    def filter_genre(song_df):
        if song_df is None:
            return None
        if genre is not None:
            i = 0
            while i < song_df.shape[0]:
                if song_df.genre[i] is None or len(intersection(genre, song_df.genre[i])) < 1:
                    song_df = song_df.drop(index = song_df.index[i], axis = 0)
                else:
                    i += 1
        return song_df[pars_weight.keys()]

    def obtain_song_candidates(my_offset, MAX_songs):
        song_df = get_songs(my_offset)
        song_df = filter_genre(song_df)
            
        #print("originally", song_df.shape[0], "song candidates", )
        if song_df is not None:
            while song_df.duration_ms.sum() / 60000 < workout_plan[-1][0] or song_df.shape[0] < MAX_songs:
                ## check total duration, if the sum of the duration of songs is smaller than the duration of the work out plan, 
                ## get more songs
                my_offset += history_args.get('limit', 20)
                new_song = get_songs(my_offset)
                if new_song is None:
                    break
                else:
                    new_song = filter_genre(new_song)
                    candidate_df = pd.concat([song_df, new_song])
                    candidate_df = candidate_df[~candidate_df.index.duplicated(keep='first')]
                if song_df.shape[0] == candidate_df.shape[0]:
                    break
                else:
                    #print("add", candidate_df.shape[0]-song_df.shape[0], "song candidates")
                    song_df = candidate_df
            if song_df.shape[0] > MAX_songs:
                song_df = song_df.iloc[range(MAX_songs), :]
            print(song_df.shape[0], "song candidates")
            return song_df
        else:
            return None
        
    def _find(all_results, curr_lst, candidates, remain_duration, trend):
        #min_length = min(candidates['duration_ms'])
        if remain_duration <= 15*1000 and remain_duration >= -15*1000:
            new_results = all_results + [curr_lst]
            print(len(new_results), 'combinations found')
            return True, new_results
        #if remain_duration < min_length - 15*1000:
        #    return False, all_results
        current_energy = get_song(user_id, token, curr_lst[-1], market)['energy'][0] if len(curr_lst) > 0 else (1 - (trend == '+'))
        candidates_df = candidates[candidates.energy > current_energy] if trend == "+" else candidates[candidates.energy <= current_energy]
        candidates_df = candidates_df[candidates_df['duration_ms'] <= remain_duration + 15000]
        candidates_df.sort_values(by=['energy'])
        found = False
        new_results = all_results
        for candidate in candidates_df.index:
            success, result = _find(new_results, curr_lst + [candidate], candidates_df.drop(index = candidate, axis = 0), 
                     remain_duration - get_song(user_id, token, candidate, market)['duration_ms'][0], trend)
            new_results = result
            if success:
                found = True
        return found, new_results

    def select_combination(all_results, energy1, energy2):
        energy_metric = lambda x: abs(x.iloc[0] - energy1) + abs(x.iloc[1] - energy2) if len(x) > 1 else abs(x.iloc[0] - energy1) + abs(x.iloc[0] - energy2)
        choice = None
        min_metric = 1
        for i in range(len(all_results)):
            xx = all_results[i]
            energy_df = pd.concat([get_song(user_id, token, id, market)['energy'] for id in xx])
            metric = energy_metric(energy_df)
            if metric < min_metric:
                min_metric = metric
                choice = i
        return(all_results[choice])
    
    def within_bucket(song_df, next_position, current_energy, next_energy, curr_lst=[]):
        trend = '+' if current_energy < next_energy else '-'
        success, result = _find(all_results=[], curr_lst=curr_lst, candidates=song_df, remain_duration=next_position, 
                                trend=trend)
        if success:
            print("all combinations found")
            return(select_combination(result, current_energy, next_energy))
        else:
            return None

    def reorder_songs(song_df, start, end, use_start = False):
        songs = []
        
        if start is not None:
            start_song = start
        else:
            max_duration = convert(workout_plan[1][0])
            df = song_df[song_df['duration_ms'] < max_duration]
            if df.shape[0] > 1:
                start_song = df.index[np.argmin((df['energy'] - workout_plan[0][1]).apply(abs))]
            else:
                start_song = song_df.index[np.argmin((song_df['energy'] - workout_plan[0][1]).apply(abs))]
        
        if end is not None:
            end_song = end
        else:
            max_duration = convert(workout_plan[-1][0] - workout_plan[-2][0])
            df = song_df[song_df['duration_ms'] < max_duration]
            if df.shape[0] > 1:
                end_song = df.index[np.argmin((df['energy'] - workout_plan[-1][1]).apply(abs))]
            else:
                end_song = song_df.index[np.argmin((song_df['energy'] - workout_plan[-1][1]).apply(abs))]
        song_df = song_df.drop(index = end_song, axis = 0) if end_song in song_df.index else song_df
        
        
        print('Start a trial!')
        current_energy = get_song(user_id, token, start_song, market)['energy'][0]
        
        for i in range(1, len(workout_plan)):
            jump_out = False
            next_position = convert(workout_plan[i][0]) - convert(workout_plan[i-1][0])
            if i == len(workout_plan)-1:
                next_position = next_position - get_song(user_id, token, end_song, market)['duration_ms'][0]
            next_energy = workout_plan[i][1]
            trend = '+' if next_energy - current_energy > 0 else '-'
            
            if use_start:
                use_start = False
                curr_lst=[start_song]
                next_position = next_position - get_song(user_id, token, start_song, market)['duration_ms'][0]
            else:
                curr_lst=[]
            print(song_df.index, 'haha', next_position, current_energy, next_energy, curr_lst)
            selected_songs = within_bucket(song_df, next_position=next_position, 
                                           current_energy=current_energy, next_energy=next_energy, 
                                           curr_lst=curr_lst)
            if selected_songs is None:
                jump_out = True
                break
            else:
                songs.append(selected_songs)
                print('Add song:', selected_songs)
                current_energy = get_song(user_id, token, selected_songs[-1], market)['energy'][0]
                song_df = song_df.drop(index = selected_songs, axis = 0)
        
        if not jump_out:
            songs.append(end_song)
            print('Add end song:', end_song)
            return songs, song_df
        else:
            return None, song_df ## no output
    
    
    start = None
    end = None
    if isinstance(start_song, str): ## store the start and end songs
        start = start_song
    if isinstance(end_song, str):
        end = end_song
        
    print("Start!")
    output = None
    my_offset = history_args.get('offset', 0)
    song_df = obtain_song_candidates(my_offset, MAX_songs)
    if song_df is None:
        raise SongError("There are no enough top songs, please change `history_args`")
    print("total of", song_df.shape[0], "song candidates")
    output, song_df = reorder_songs(song_df, start, end, use_start = True)
    if output is not None and len(output) > 1:
        output = flatten(output)
        print("got a target list", output)
        return output
    else:
        print("after 1 trial")
        while output is None:
            my_offset += MAX_songs
            MAX_songs += MAX_songs
            print("offset, limit", my_offset, MAX_songs)
            new_df = obtain_song_candidates(my_offset, MAX_songs)
            if new_df is not None:
                candidate_df = pd.concat([song_df, new_df])
                candidate_df = candidate_df[~candidate_df.index.duplicated(keep='first')]
                if song_df.shape[0] == candidate_df.shape[0]:
                    break ## no new song candidates to add
                else:
                    song_df = candidate_df.copy()
            else: ## no new song candidates to add
                break
            print("total of", song_df.shape[0], "song candidates")
            if song_df is not None:
                output, song_df = reorder_songs(song_df, start, end, use_start = False)
                output = flatten(output)
                print("got a target list", output)
                return output
        raise SongError("There are avaliable songs, please change `history_args` or `workout_plan`")

### Print a table for different `time_range`


- long_term (calculated from several years of data and including all new data as it becomes available), 
- medium_term (approximately last 6 months), 
- short_term (approximately last 4 weeks)

In [517]:
def get_song_name(user_id, token, song_id, market):
    
    ## get album information and the first artist
    base_url = "https://api.spotify.com/v1/tracks/"
    query = f'{base_url}{song_id}?market={market}'
    response = requests.get(query, 
                   headers={"Accept": "application/json",
                            "Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})
    json_response = response.json()
    df_response = pd.json_normalize(json_response)
    return df_response[['name', 'id']].set_index('id')


def print_table(user_id, token, market, song_num = 150, history_args = {}, genre = None):

    intersection = lambda lst1, lst2: [value for value in lst1 if value in lst2]
    def get_songs(my_offset):
        offset = max(my_offset, history_args.get('offset', 0))
        history_args['offset'] = offset
        song_ids = get_top_songs(user_id, token, **history_args)
        if len(song_ids) > 0:
            #song_ids = set(song_ids) ## remove duplicate songs
            song_df = pd.concat([get_song(user_id, token, id, market) for id in song_ids])
            return song_df
        else:
            return None
    
    def filter_genre(song_df):
        if song_df is None:
            return None
        if genre is not None:
            i = 0
            while i < song_df.shape[0]:
                if song_df.genre[i] is None or len(intersection(genre, song_df.genre[i])) < 1:
                    song_df = song_df.drop(index = song_df.index[i], axis = 0)
                else:
                    i += 1
        return song_df

    my_offset = history_args.get('offset', 0)
    song_df = get_songs(my_offset)
    song_df = filter_genre(song_df)
    if song_df is None:
        raise SongError("There are no enough top songs, please change `history_args`")
        
    while song_df.shape[0] < song_num + history_args.get('limit', 20):
        my_offset += history_args.get('limit', 20)
        new_song = get_songs(my_offset)
        if new_song is None:
            break
        else:
            new_song = filter_genre(new_song)
            candidate_df = pd.concat([song_df, new_song])
            candidate_df = candidate_df[~candidate_df.index.duplicated(keep='first')]
        if song_df.shape[0] == candidate_df.shape[0]:
            break
        else:
            song_df = candidate_df
    if song_df.shape[0] > song_num:
        song_df = song_df.iloc[range(song_num), :]
    print("finally", song_df.shape[0], "song candidates")
    
    table = song_df['genre']
    aa = pd.DataFrame.from_dict(Counter(table.index), orient='index').reset_index().rename(columns={'index':'id', 0:'frequency'}).set_index('id')
    dff = pd.concat([get_song_name(user_id, token, idx, market = 'US') for idx in aa.index])
    return pd.merge((pd.merge(aa, dff, how = 'left', on = 'id')), table, how = 'left', on = 'id')

# Test

In [515]:
# settings
user_id = "pbwppse1hilahmk43ls424ao4" # "2272ss2bp3tznxg6k3v63kqxq" 
token = "BQB590DROWL4KPqi_mmtMcQi3vb5qrrfTKCY3cIhP4gCOB2WXAMIeSylLMX7aK1dHbQGoizM5NqgDHN9aUIV4a6kBCdGhic8X81a0gLFvvg5daXhrYRomlxv4quOxntPZMG5iLJ7BC1n8KNrvSlNrwNByu4hE6RRpdkkpD_PNZoPHPRGW6SdU7R2x39SWcoBnVQYbDMr3y2bcQ0p1LLANP0RcUuqKFbOnldYzBF8xfvYe5GQs5u5lqmCHw"

#### See tables with top songs and frequency

In [228]:
term = 'short_term'
history_args = {'limit': 10, 'time_range': term, 'offset': 0}
table = print_table(user_id, token, market, song_num = 150, history_args = history_args)
table

status: 200
status: 200
status: 200
status: 200
finally 26 song candidates


,frequency,name,genre
id,,,
09RXXnMSVrES6xju7IXrsX,1,Proof - Lifelike Remix,None
0HDudjyoG3UpWIMAJrXIaB,1,El Sueño - Radio Edit,"[house, latin tech house]"
0MjeQ5T5E50q0d3i72854N,1,I Like to Move it - Radio Mix,None
13EMVBU1hcy5yJdX1Xhf9a,1,Ghostkeeper,"[austrian pop, deep house, new french touch, t..."
18MlLrfCkYOnKutil99GwY,1,Alter Ego,"[minimal melodic techno, minimal techno, new f..."
1oWNMCtrpGV71z6dWRQAWV,1,I've Been Thinking About You,"[deep groove house, funky house, funky tech ho..."
1uU95PBinoqgQVn6VVpF6q,1,La Vereda (Cada Vez),"[funky tech house, italian tech house]"
1uXUZfJykefWuwj9VtyxIq,1,Trauma - Worakls Remix,"[minimal melodic techno, minimal techno, new f..."
1wu9kYSeWm0txCdISRcmHo,1,Could You Be Loved [Club Mix],[funky tech house]


In [229]:
term = 'medium_term'
history_args = {'limit': 10, 'time_range': term, 'offset': 0}
table1 = print_table(user_id, token, market, song_num = 150, history_args = history_args)
table1

status: 200
status: 200
status: 200
status: 200
finally 28 song candidates


,frequency,name,genre
id,,,
09RXXnMSVrES6xju7IXrsX,1,Proof - Lifelike Remix,None
7bzks4LGpQUuPKBzJ6iQ7y,1,Playground,"[french indie pop, french indietronica, new fr..."
7nBR4Tt431p1MTgv3lVsmX,1,Get Your Thing Together - Soulmagic Main Mix,[funky house]
0HDudjyoG3UpWIMAJrXIaB,1,El Sueño - Radio Edit,"[house, latin tech house]"
0MjeQ5T5E50q0d3i72854N,1,I Like to Move it - Radio Mix,None
13EMVBU1hcy5yJdX1Xhf9a,1,Ghostkeeper,"[austrian pop, deep house, new french touch, t..."
18MlLrfCkYOnKutil99GwY,1,Alter Ego,"[minimal melodic techno, minimal techno, new f..."
1oWNMCtrpGV71z6dWRQAWV,1,I've Been Thinking About You,"[deep groove house, funky house, funky tech ho..."
1uU95PBinoqgQVn6VVpF6q,1,La Vereda (Cada Vez),"[funky tech house, italian tech house]"


In [230]:
term = 'long_term'
history_args = {'limit': 10, 'time_range': term, 'offset': 0}
table2 = print_table(user_id, token, market, song_num = 150, history_args = history_args)
table2

status: 200
status: 200
status: 200
status: 200
finally 28 song candidates


,frequency,name,genre
id,,,
09RXXnMSVrES6xju7IXrsX,1,Proof - Lifelike Remix,None
7bzks4LGpQUuPKBzJ6iQ7y,1,Playground,"[french indie pop, french indietronica, new fr..."
7nBR4Tt431p1MTgv3lVsmX,1,Get Your Thing Together - Soulmagic Main Mix,[funky house]
0HDudjyoG3UpWIMAJrXIaB,1,El Sueño - Radio Edit,"[house, latin tech house]"
0MjeQ5T5E50q0d3i72854N,1,I Like to Move it - Radio Mix,None
13EMVBU1hcy5yJdX1Xhf9a,1,Ghostkeeper,"[austrian pop, deep house, new french touch, t..."
18MlLrfCkYOnKutil99GwY,1,Alter Ego,"[minimal melodic techno, minimal techno, new f..."
1oWNMCtrpGV71z6dWRQAWV,1,I've Been Thinking About You,"[deep groove house, funky house, funky tech ho..."
1uU95PBinoqgQVn6VVpF6q,1,La Vereda (Cada Vez),"[funky tech house, italian tech house]"


#### Test top songs without genre

In [518]:
market = 'US'
workout_plan = [(0, 0.6), (15, 0.95), (23, 0.25)]
pars_weight = {'energy': 0.65, 'duration_ms': 0.35}
history_args = {'limit': 15, 'time_range': 'long_term', 'offset': 0}

In [519]:
songs = generate_target(user_id, token, market, workout_plan, history_args, 
                    pars_weight = pars_weight, recently = False, genre = None, MAX_songs = 25)
songs

Start!
status: 200
status: 200
25 song candidates
total of 25 song candidates
Start a trial!
Index(['09RXXnMSVrES6xju7IXrsX', '1uXUZfJykefWuwj9VtyxIq',
       '7bzks4LGpQUuPKBzJ6iQ7y', '7nBR4Tt431p1MTgv3lVsmX',
       '0MjeQ5T5E50q0d3i72854N', '13EMVBU1hcy5yJdX1Xhf9a',
       '1uU95PBinoqgQVn6VVpF6q', '1wu9kYSeWm0txCdISRcmHo',
       '1oWNMCtrpGV71z6dWRQAWV', '2riAqGVmeUIwoB8rjxo3Tt',
       '0HDudjyoG3UpWIMAJrXIaB', '18MlLrfCkYOnKutil99GwY',
       '2KklXplRtxMsBYo474Es0w', '3iBvt1v6CbsNL0FMhDOkzk',
       '7BYgNW3S6AjRT0AWuFFlwu', '5mjYbfPaXes4osPzyXNGfL',
       '4jHB8cLwEtbFzcbwymr0Do', '3kZqrLzuoE54EfHPwappr6',
       '70M59uUigQFmXYImTnqjGp', '68Q6c22JNyLUlZYX4dE2tn',
       '4DIspILFDWAlU7bViTMqhS', '4xxyCh7xkK30LVBO81poif',
       '55bfXeq8wXtyKQtSZB3KYF', '6ldJYmlEQYPPBmoxZjpMvr'],
      dtype='object', name='id') haha 602015 0.584 0.95 ['70M59uUigQFmXYImTnqjGp']
1 combinations found
2 combinations found
3 combinations found
4 combinations found
5 combinations found
6 combinat

['70M59uUigQFmXYImTnqjGp',
 '0MjeQ5T5E50q0d3i72854N',
 '7BYgNW3S6AjRT0AWuFFlwu',
 '4xxyCh7xkK30LVBO81poif',
 '2KklXplRtxMsBYo474Es0w',
 '29Q7DaDXdIibBLCivJwIqA']

In [522]:
durations = pd.concat([get_song(user_id, token, id, market)['duration_ms'] for id in songs])
durations/ 60/ 1000, durations.sum() / 60/ 1000

(id
 70M59uUigQFmXYImTnqjGp    4.966417
 0MjeQ5T5E50q0d3i72854N    2.641683
 7BYgNW3S6AjRT0AWuFFlwu    7.344333
 4xxyCh7xkK30LVBO81poif    2.770183
 2KklXplRtxMsBYo474Es0w    2.688700
 29Q7DaDXdIibBLCivJwIqA    2.716717
 Name: duration_ms, dtype: float64,
 23.128033333333335)

In [523]:
pd.concat([get_song(user_id, token, id, market)['energy'] for id in songs])

id
70M59uUigQFmXYImTnqjGp    0.584
0MjeQ5T5E50q0d3i72854N    0.929
7BYgNW3S6AjRT0AWuFFlwu    0.961
4xxyCh7xkK30LVBO81poif    0.908
2KklXplRtxMsBYo474Es0w    0.666
29Q7DaDXdIibBLCivJwIqA    0.321
Name: energy, dtype: float64

Create a playlist with those songs

In [63]:
create_plst(user_id, token, ['spotify:track:'+ str(x) for x in songs], 
            name='Top no genre', description='', public=False)

Playlist Top no genre is successfully created!
Playlist Top no genre is successfully filled with recommendations!
Your playlist is ready at https://open.spotify.com/playlist/76cVJo7Hnkz58NESXL2tEu


#### Test top songs with genre

In [524]:
workout_plan = [(0, 0.6), (15, 0.95), (23, 0.25)]

history_args = {'limit': 15, 'time_range': 'long_term', 'offset': 0}

genre = ['new french touch', 'trip hop', 'funky house']

In [525]:
songs = generate_target(user_id, token, market, workout_plan, history_args, 
                    pars_weight = pars_weight, recently = False, genre = genre, MAX_songs = 25)
songs

Start!
status: 200
status: 200
status: 200
13 song candidates
total of 13 song candidates
Start a trial!
Index(['1uXUZfJykefWuwj9VtyxIq', '7bzks4LGpQUuPKBzJ6iQ7y',
       '7nBR4Tt431p1MTgv3lVsmX', '13EMVBU1hcy5yJdX1Xhf9a',
       '1oWNMCtrpGV71z6dWRQAWV', '2riAqGVmeUIwoB8rjxo3Tt',
       '2KklXplRtxMsBYo474Es0w', '3iBvt1v6CbsNL0FMhDOkzk',
       '7BYgNW3S6AjRT0AWuFFlwu', '4jHB8cLwEtbFzcbwymr0Do',
       '6ldJYmlEQYPPBmoxZjpMvr', '61DZiQc2sRohLFApD8lTSk'],
      dtype='object', name='id') haha 612926 0.552 0.95 ['6ldJYmlEQYPPBmoxZjpMvr']
1 combinations found
2 combinations found
3 combinations found
all combinations found
Add song: ['6ldJYmlEQYPPBmoxZjpMvr', '7nBR4Tt431p1MTgv3lVsmX', '4jHB8cLwEtbFzcbwymr0Do']
Index(['1uXUZfJykefWuwj9VtyxIq', '7bzks4LGpQUuPKBzJ6iQ7y',
       '13EMVBU1hcy5yJdX1Xhf9a', '1oWNMCtrpGV71z6dWRQAWV',
       '2riAqGVmeUIwoB8rjxo3Tt', '2KklXplRtxMsBYo474Es0w',
       '3iBvt1v6CbsNL0FMhDOkzk', '7BYgNW3S6AjRT0AWuFFlwu',
       '61DZiQc2sRohLFApD8lTSk'],
      dtype=

SongError: There are avaliable songs, please change `history_args` or `workout_plan`

In [ ]:
durations = pd.concat([get_song(user_id, token, id, market)['duration_ms'] for id in songs])
durations/ 60/ 1000, durations.sum() / 60/ 1000

In [ ]:
pd.concat([get_song(user_id, token, id, market)['energy'] for id in songs])

Create a playlist with those songs

In [ ]:
create_plst(user_id, token, ['spotify:track:'+ str(x) for x in songs], 
            name='Top with a genre', description='', public=False)

======================================================================================================================

#### Test recently played songs without genre

In [ ]:
# token = "BQC7ERke6mhHX9C_-OnMGgDkMl4cGwomSWB2L9ENErXgnwAHJDpjCf12vH_ZIJ3mJIdyLt1z8_vmqFTgmFQCdedB0Nx5uqNL-vkYmv3J5eJqfUgABRkFPssOpQMzPenpYNki5vEEXLb3aB2HEGnOBOGNgBEK11N0h0ALB8ZU8EP18ZmDfLprSrVlbldSCt-ir6K9VplOSkI5rDktXFtj-MX7WGkgiBA-LIaYa0XOFsL1qyBPZSE8Uhn4dA"

In [56]:
#market = 'US'
#workout_plan = [(0, 0.6), (15, 0.95), (30, 0.8), (35, 0.25)]
#pars_weight = {'energy': 0.65, 'duration_ms': 0.35}

#history_args = {'limit': 7, 'after': 0} ## {} means using default values

In [ ]:
#songs = generate_target(user_id, token, market, workout_plan, history_args, pars_weight = pars_weight, 
#                        recently = True, MAX_songs = 50)
#songs

See the duration for each song and the total duration

In [ ]:
#durations = pd.concat([get_song(user_id, token, id, market)['duration_ms'] for id in songs])
#durations / 60/ 1000, durations.sum() / 60/ 1000

See the energy for each song

In [ ]:
#pd.concat([get_song(user_id, token, id, market)['energy'] for id in songs])

Create a playlist with those songs

In [ ]:
#create_plst(user_id, token, ['spotify:track:'+ str(x) for x in songs], 
#            name='Recently no genre', description='', public=False)

#### Test recently played songs with genre

You can go to https://developer.spotify.com/console/get-available-genre-seeds/ to find all avaliable genres.

In [ ]:
#pars_weight = {'energy': 0.8, 'duration_ms': 0.2}

#history_args = {'limit': 15, 'after': 28885117} ## {} means using default values

#genre = ['disco', 'deep house', 'disco house', 'funky house', 'groove room']

In [ ]:
#songs = generate_target(user_id, token, market, workout_plan, history_args, 
#                    pars_weight = pars_weight, genre = genre)
#songs

In [ ]:
#durations = pd.concat([get_song(user_id, token, id, market)['duration_ms'] for id in songs])
#durations/ 60/ 1000, durations.sum() / 60/ 1000

In [ ]:
#pd.concat([get_song(user_id, token, id, market)['energy'] for id in songs])

Create a playlist with those songs

In [ ]:
#create_plst(user_id, token, ['spotify:track:'+ str(x) for x in songs], 
#            name='Recently with a genre', description='', public=False)